<a href="https://colab.research.google.com/github/TheHackerLlama/charlas/blob/main/riiaa_2021/parte1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install sentencepiece
!pip install transformers

## Análisis de sentimiento

In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
text = "I'm very happy to be in this workshop"
classifier(text)

In [ ]:
import pandas as pd

outputs = classifier(text)
pd.DataFrame.from_records(outputs)

In [ ]:
examples = ["I'm very happy to be in this workshop", "I'm bored and want to sleep"]
outputs = classifier(examples)
pd.DataFrame.from_records(outputs)

In [ ]:
text = """Dear Amazon, last week I ordered an Optimus Prime action figure \
from your online store in Germany. Unfortunately, when I opened the package, \
I discovered to my horror that I had been sent an action figure of Megatron \
instead! As a lifelong enemy of the Decepticons, I hope you can understand my \
dilemma. To resolve the issue, I demand an exchange of Megatron for the \
Optimus Prime figure I ordered. Enclosed are copies of my records concerning \
this purchase. I expect to hear from you soon. Sincerely, Bumblebee."""

In [ ]:
outputs = classifier(text)
pd.DataFrame.from_records(outputs)

## Reconocimiento de Entidades Nombradas (Named Entity Recognition)

In [ ]:
ner_tagger = pipeline("ner", aggregation_strategy="simple")
outputs = ner_tagger(text)
pd.DataFrame.from_records(outputs)

## Responder Preguntas - Question Answering

In [ ]:
question_answerer = pipeline("question-answering")
question_answerer(
    question="Where do I work?",
    context="My name is Omar and I work at Hugging Face in Zurich"
)

In [ ]:
question = "What does the customer want?"
outputs = question_answerer(question=question, context=text)
pd.DataFrame.from_records([outputs])

## Análisis de Sentimiento en Español

Podemos buscar modelos en https://huggingface.co/models?language=es&pipeline_tag=text-classification&sort=downloads

In [ ]:
classifier = pipeline("sentiment-analysis", model="daveni/twitter-xlm-roberta-emotion-es")
pd.DataFrame.from_records(classifier("Hoy estoy muy feliz "))

In [ ]:
pd.DataFrame.from_records(classifier(["Hoy estoy muy feliz", "Estoy feliz", "Creo que el diseño está algo viejo"]))

# Traducción

In [ ]:
translator = pipeline("translation_en_to_es", 
                      model="Helsinki-NLP/opus-mt-en-es")
translated_output = translator(text, min_length=100)[0]["translation_text"]
print(translated_output)

## Resumir / Summarization

In [ ]:
summarizer = pipeline("summarization")
outputs = summarizer(text, max_length=45, clean_up_tokenization_spaces=True)
print(outputs[0]['summary_text'])

## Generación de texto

In [ ]:
response = "Dear Bumblebee, I am sorry to hear that your order was mixed up."
prompt = text + "\n\nCustomer service response:\n" + response
print(prompt)

In [ ]:
from transformers import set_seed
set_seed(42)

generator = pipeline("text-generation")
outputs = generator(prompt, max_length=200)
print(outputs[0]['generated_text'])

In [ ]:
generator = pipeline("text-generation", model="datificate/gpt2-small-spanish")
response = "Querido Bumblebee, lamento escuchar que tu órden no fue la correcta."
prompt = translated_output + "\n\nRespuesta servicio al cliente:\n" + response
outputs = generator(prompt, max_length=400)
print(outputs[0]['generated_text'])

## Sesgos en el modelo

In [ ]:
unmasker = pipeline("fill-mask", model="bert-base-uncased")
result = unmasker("This man works as a [MASK].")
print([r["token_str"] for r in result])

result = unmasker("This woman works as a [MASK].")
print([r["token_str"] for r in result])